In [1]:
from util import *

flag='small'
playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,playlist_tracks_tru,tracks,playlists,playlist_tracks = load_small_data()
y_true=ytrue(playlists_tst,playlist_tracks_tru)

playlist_vecs=np.load('raw/w2v/playlist_vecs.npy')
track_vecs=np.load('raw/w2v/track_vecs.npy')

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

In [2]:
track_vecs_tra=track_vecs[tracks.track_id]
tid_tra=tracks.track_id.values
emb_dim=track_vecs.shape[1]

In [3]:
from keras.layers.embeddings import Embedding

def build_embedding_layer():
    vecs=np.c_.concatenate([playlist_vecs,track_vecs])
    len_dim=vecs.shape[0]+1
    emb_dim=vecs.shape[1]
    
    emb_matrix=np.zeros((len_dim,emb_dim))
    emb_matrix[:-1,:]=vecs
    
    emb_layer=Embedding(len_dim,emb_dim,trainable=False)
    emb_layer.build((None,))
    emb_layer.set_weights([emb_matrix])
    
    return emb_layer

embedding_layer = build_embedding_layer()
assert abs(embedding_layer.get_weights()[0][1][3] - playlist_vecs[1][3])<1e-3

/home/cq/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [157]:
import numpy as np
np.random.seed=0
from keras.models import Model,Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, SimpleRNN, InputLayer, Lambda, Reshape
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed=1
max_len=100

In [160]:
X=np.zeros((playlists_tra.shape[0],max_len),dtype='int')
Y=np.zeros((playlists_tra.shape[0],max_len,emb_dim))

for i in tnrange(playlists_tra.shape[0]):
    pid=playlists_tra.iloc[i]['pid']
    X[i][0]=pid
    tracks=playlist_tracks_tra[playlist_tracks_tra.pid==pid].track_id.values
    for j in range(max_len-1):
        if j<tracks.shape[0]:
            X[i][j+1]=tracks[j]+playlist_vecs.shape[0]
            Y[i][j]=track_vecs[tracks[j]]
    if j < tracks.shape[0]-1:
        Y[i][j+1]=track_vecs[tracks[j+1]]
Y=Y.reshape(Y.shape[1],Y.shape[0],Y.shape[2])

HBox(children=(IntProgress(value=0, max=9900), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [161]:
import numpy as np
def l2_norm_2(x, axis=None):
    """
    takes an input tensor and returns the l2 norm along specified axis
    """

    square_sum = np.sum(np.square(x), axis=axis, keepdims=True)
    norm = np.sqrt(np.maximum(square_sum, 1e-4))

    return norm

x=np.random.rand(50,1)
a=np.dot(track_vecs,x)/(l2_norm_2(track_vecs,axis=1)*l2_norm_2(x))
np.argmax(a)

364043

In [169]:
from keras import backend as K
from keras.engine.topology import Layer


class PidLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(PidLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel=K.constant(track_vecs,name='track_vecs')
        super(PidLayer, self).build(input_shape)  # Be sure to call this at the end
        

    def call(self, x):
#         return K.argmax(K.dot(self.kernel,x)/(l2_norm(self.kernel,axis=1) * l2_norm(x)))
        return K.argmax(K.dot(self.kernel,x))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

class PlaylistGenerator:
    def __init__(self,n_a,emb_dim,max_len):
        self.n_a = n_a
        X=Input(shape=(max_len,))
        a0=Input(shape=(n_a,))
        c0=Input(shape=(n_a,))
        outputs = []

        reshapor1=Reshape((1,emb_dim),name='reshape1')
        reshapor2=Reshape((emb_dim,1),name='reshape2')
        self.rnn_cell = LSTM(n_a,return_state=True)
        self.densor=Dense(emb_dim)

        a = a0
        c = c0
        for i in range(max_len):
            x=Lambda(lambda x:x[:,i])(X)
            x=embedding_layer(x)
            
            x=reshapor1(x)
            
            a,_,c=self.rnn_cell(x,initial_state=[a,c])
            o=self.densor(a)
            outputs.append(o)

        self.model_train=Model(inputs=[X,a0],outputs=outputs)
#         self.model_train.summary()
        
        self.model_train.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
        
        x0 = Input(shape=(emb_dim,))
        a0 = Input(shape=(n_a,),name='a0')
        c0 = Input(shape=(n_a,),name='c0')
        a = a0
        c = c0
        x = x0
        
        pid_layer=PidLayer(1,trainable=False)
        pid_layer.build((emb_dim,))
        
        x=reshapor1(x)
        a,_=self.rnn_cell(x,initial_state=[a,c])
        o=self.densor(a)
            
        self.model_infer=Model(inputs=[x0,a0],outputs=[o,a])
#         self.model_infer.summary()

        self.tv_l2=l2_norm_2(track_vecs_tra,axis=1)
        
    def fit(self,X,Y,epochs):
        a0=np.zeros((X.shape[0],self.n_a))
        self.model_train.fit([X,a0],list(Y),epochs=epochs,batch_size=64)
        
        
        
    def predict(self,X):
        a=np.zeros((1,self.n_a))
        
        results=[]
        for i in range(len(X)):
            tv,a=self.model_infer.predict([track_vecs[X[i]].reshape(1,emb_dim),a])
            
        if len(X)>1:
            tv=track_vecs[X[-1]]
            
        for i in range(500):
            tv,a=self.model_infer.predict([tv.reshape(1,emb_dim),a])
            idxs=np.argsort((np.dot(track_vecs_tra,tv.T)/(self.tv_l2*l2_norm_2(tv.T))).reshape((track_vecs_tra.shape[0],)))
#             print(idxs[-10:])
            for j in range(len(idxs)-1,-1,-1):
                tid=tid_tra[idxs[j]]
                if tid not in results and tid not in X:
                    results.append(tid)
                    tv=track_vecs[tid]
                    break
            
                    
        return results
    
n_a=128
model=PlaylistGenerator(n_a,emb_dim,max_len)
    

ValueError: An `initial_state` was passed that is not compatible with `cell.state_size`. Received `state_spec`=[InputSpec(shape=(None, 128), ndim=2)]; however `cell.state_size` is (128, 128)

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [ ]:
model.fit(X,Y,epochs=5)

In [ ]:
y_pred={}
for i in tnrange(playlists_tst.shape[0]):
    pid=playlists_tst.iloc[i]['pid']
    x=[pid]
    track_ids=playlist_tracks_tst[playlist_tracks_tst.pid==pid].track_id.values
    x+=list(track_ids)
    y=model.predict(x)
#     print(y[:10])
    y_pred[pid]=y

In [ ]:
assert y_pred.keys()==y_true.keys()
print('r_prec %.3f, ndcg %.3f, click %.3f'%(r_prec(y_true,y_pred),ndcg(y_true,y_pred),clicks(y_true,y_pred)))

In [110]:
y_pred.keys()

dict_keys([90112, 427008, 90114, 90115, 90116, 417797, 427009, 427011, 427012, 417801, 90122, 417802, 427013, 427015, 427017, 90127, 427018, 417809, 427020, 427021, 417812, 90133, 427022, 427023, 427024, 427025, 427026, 427027, 427029, 417821, 427030, 427032, 427034, 427035, 417826, 427036, 427037, 427038, 427044, 417831, 417832, 417833, 90154, 427010, 427045, 427046, 427047, 427048, 427049, 427050, 427051, 427053, 417844, 427056, 90166, 427057, 427060, 427061, 427063, 427059, 427062, 427064, 427065, 427014, 427066, 427067, 427068, 427069, 427070, 427071, 427072, 427073, 427074, 427016, 427075, 427076, 427077, 427078, 427079, 427019, 427028, 427031, 427033, 427039, 427040, 427041, 427042, 427043, 427052, 427054, 427055, 427058, 427000, 427001, 427002, 427003, 427004, 427005, 427006, 427007])

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [166]:
y_pred[90114][:10]

[523556,
 1286544,
 562969,
 727447,
 516688,
 1752936,
 2139820,
 155159,
 1877192,
 1769567]

In [168]:
y_pred[417812][:10]

[1286544,
 523556,
 727447,
 562969,
 516688,
 1752936,
 155159,
 2139820,
 1877192,
 1769567]

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
